In [2]:
import pandas as pd
from datetime import datetime as dt
import datetime as dat
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
jobs = pd.read_csv("../data/fullsample.csv", nrows=10000)

In [4]:
ce5 = pd.read_csv('../data/slurm_wrapper_ce5.log',
                  header=None,
                  delimiter=' - ',
                  engine='python',
                  nrows=10000)

In [5]:
ce6 = pd.read_csv('../data/slurm_wrapper_ce6.log',
                  header=None,
                  delimiter=' - ',
                  engine='python',
                  nrows=10000)

In [6]:
colnames = ['END', 'USER', 'RETRY', 'TIME', 'RETURNCODE', 'COMMAND']
ce5.columns=colnames
ce6.columns=colnames

In [7]:
ce5['END'] = [dt.strptime(row + ('.000000' if '.' not in row else ''), '%Y-%m-%d %H:%M:%S.%f') for row in ce5['END']]
ce5['TIME'] = ce5['TIME'].str.replace('time', '', regex=False).astype(float)
ce5['USER'] = ce5['USER'].str.replace('user', '', regex=False).astype(int)
ce5['RETRY'] = ce5['RETRY'].str.replace('retry', '', regex=False).astype(int)
ce5['RETURNCODE'] = ce5['RETURNCODE'].str.replace('returncode', '', regex=False).astype(float)
ce5['COMMAND'] = ce5['COMMAND'].str.replace('command', '', regex=False)

In [8]:
ce6['END'] = [dt.strptime(row + ('.000000' if '.' not in row else ''), '%Y-%m-%d %H:%M:%S.%f') for row in ce6['END']]
ce6['TIME'] = ce6['TIME'].str.replace('time', '', regex=False).astype(float)
ce6['USER'] = ce6['USER'].str.replace('user', '', regex=False).astype(int)
ce6['RETRY'] = ce6['RETRY'].str.replace('retry', '', regex=False).astype(int)
ce6['RETURNCODE'] = ce6['RETURNCODE'].str.replace('returncode', '', regex=False).astype(float)
ce6['COMMAND'] = ce6['COMMAND'].str.replace('command', '', regex=False)

In [9]:
jobs.head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0


In [10]:
jobs = jobs.replace('Unknown', np.nan)

In [11]:
jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'])
jobs['END'] = pd.to_datetime(jobs['END'])

In [12]:
jobs

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31 22:15:00,NaT,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06 11:36:09,2021-09-05 11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06 19:04:39,2021-09-05 19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09 16:52:51,2021-09-07 20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17 07:45:07,2021-09-10 16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,31874232_4806,COMPLETED,2021-09-01 00:09:08,2021-09-01 00:15:22,2048Mn,146.12M,00:15:00,00:06:14,1,1,production,0:0
9996,31874232_4807,COMPLETED,2021-09-01 00:09:08,2021-09-01 00:15:26,2048Mn,150.64M,00:15:00,00:06:18,1,1,production,0:0
9997,31874232_4808,COMPLETED,2021-09-01 00:09:08,2021-09-01 00:15:05,2048Mn,146.44M,00:15:00,00:05:57,1,1,production,0:0
9998,31874232_4809,COMPLETED,2021-09-01 00:09:08,2021-09-01 00:15:00,2048Mn,146.60M,00:15:00,00:05:52,1,1,production,0:0


In [41]:
jobs[jobs['END'] == '2020-10-16']

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE


In [14]:
ce6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   END         10000 non-null  datetime64[ns]
 1   USER        10000 non-null  int32         
 2   RETRY       10000 non-null  int32         
 3   TIME        10000 non-null  float64       
 4   RETURNCODE  10000 non-null  float64       
 5   COMMAND     10000 non-null  object        
dtypes: datetime64[ns](1), float64(2), int32(2), object(1)
memory usage: 390.8+ KB


In [15]:
ce6.loc[ce6['TIME'] >= 14.00]

,END,USER,RETRY,TIME,RETURNCODE,COMMAND
76,2020-10-16 10:47:25.629251,9202,0,14.706054,0.0,"['/usr/bin/scontrol', 'show', 'job']"
86,2020-10-16 10:48:47.889628,9202,0,18.981121,0.0,"['/usr/bin/scontrol', 'show', 'job']"
90,2020-10-16 10:48:51.353412,9219,0,18.934414,0.0,"['/usr/bin/scontrol', 'show', 'job']"
93,2020-10-16 10:50:03.211199,9203,0,17.303350,0.0,"['/usr/bin/scontrol', 'show', 'job']"
98,2020-10-16 10:51:21.419356,9202,0,14.691301,0.0,"['/usr/bin/scontrol', 'show', 'job']"
...,...,...,...,...,...,...
9581,2020-10-16 21:14:00.436654,9219,0,24.916284,0.0,"['/usr/bin/scontrol', 'show', 'job']"
9582,2020-10-16 21:14:00.795042,9204,0,23.679522,0.0,"['/usr/bin/scontrol', 'show', 'job']"
9732,2020-10-16 21:22:21.575417,9219,0,14.017365,0.0,"['/usr/bin/scontrol', 'show', 'job']"
9856,2020-10-16 21:28:31.289137,9203,0,16.902042,0.0,"['/usr/bin/scontrol', 'show', 'job']"


In [16]:
jobs['END'] = dt.now().strftime("%Y-%m-%dT%H:%M:%S")

In [17]:
jobs[jobs['END'] == 2020-10-16]

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
